In [113]:
import sys, os

root = r'C:\Users\user\Documents\Fine-VLA'

if root not in sys.path:
    sys.path.insert(0, root)

os.chdir(root)

import data_loader.data_loader as module_data
import model.metric as module_metric
import model.model as module_arch
from model.model import compute_similarity
from parse_config import ConfigParser
from trainer.trainer import verbose
from utils.util import state_dict_data_parallel_fix

In [114]:
import argparse

import pandas as pd
import torch
import transformers
from sacred import Experiment
from tqdm import tqdm
import glob

import numpy as np
import os
import copy
import pathlib
import platform

In [115]:
import sys

original_argv = sys.argv.copy()
sys.argv = ['ipython']

parser = argparse.ArgumentParser(description='PyTorch Template')

parser.add_argument('-r', '--resume', default=None, type=str,
                    help='path to latest checkpoint (default: None)')
parser.add_argument('-d', '--device', default=None, type=str,
                    help='indices of GPUs to enable (default: all)')
parser.add_argument('-c', '--config', default=None, type=str,
                    help='config file path (default: None)')
parser.add_argument('-s', '--sliding_window_stride', default=-1, type=int,
                    help='test time temporal augmentation, repeat samples with different start times.')
parser.add_argument('--save_feats', default=None,
                    help='path to store text & video feats, this is for saving embeddings if you want to do offline retrieval.')
parser.add_argument('--save_type', default='both', choices=['both', 'text', 'video'],
                    help='Whether to save video, text or both feats. If running on inference videos, text is just a placeholder')
parser.add_argument('--vis_token_similarity', action='store_true')
parser.add_argument('--split', default='test', choices=['train', 'val', 'test'],
                    help='split to evaluate on.')
parser.add_argument('--batch_size', default=16, type=int,
                    help='size of batch')

_StoreAction(option_strings=['--batch_size'], dest='batch_size', nargs=None, const=None, default=16, type=<class 'int'>, choices=None, required=False, help='size of batch', metavar=None)

In [116]:
sys.argv.extend([
    '-c', 'configs/ntu.json',
    '-r', 'exps/pretrained/cc-webvid2m-4f_stformer_b_16_224.pth.tar',
    '--split', 'test',
    '--batch_size', '16'
])
print(sys.argv)


['ipython', '-c', 'configs/ntu.json', '-r', 'exps/pretrained/cc-webvid2m-4f_stformer_b_16_224.pth.tar', '--split', 'test', '--batch_size', '16']


In [ ]:
config = ConfigParser(parser, test=True)

print(f"Dataset name: {config['data_loader']['args']['dataset_name']}")
print(f"Data dir: {config['data_loader']['args']['data_dir']}")
print(f"Cut: {config['data_loader']['args']['cut']}")

args = parser.parse_args()

Dataset name: NTU
Data dir: data/nturgbd_rgb
Cut: standard


In [106]:
data_loader = config.initialize('data_loader', module_data)

""" 이거 한번 만들어보자"""

TextVideoDataLoader


FileNotFoundError: [Errno 2] No such file or directory: 'data/nturgbd_rgb\\annotations.csv'

In [73]:
module_name = config['data_loader']['type']
module_args = dict(config['data_loader']['args'])

print(module_name)
print(module_args)

TextVideoDataLoader
{'dataset_name': 'NTU', 'data_dir': 'data/nturgbd_rgb', 'shuffle': True, 'num_workers': 16, 'batch_size': 16, 'split': 'train', 'cut': 'standard', 'subsample': 1, 'text_params': OrderedDict([('input', 'text')]), 'video_params': OrderedDict([('extraction_fps', 25), ('extraction_res', 256), ('input_res', 224), ('num_frames', 4), ('stride', 1)])}


In [76]:
NTU(module_args)

TypeError: TextVideoDataset.__init__() missing 3 required positional arguments: 'text_params', 'video_params', and 'data_dir'